In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tqdm

In [2]:
def cm_to_iou(cm):
    tps = np.diag(cm)
    fps = cm.sum(axis=0) - tps
    fns = cm.sum(axis=1) - tps
    iou = tps / (tps + fps + fns)
    return iou.squeeze()

def cm_to_miou(cm):
    iou = cm_to_iou(cm)
    return iou.mean()

def read_metrics(metrics_path):
    with open(metrics_path, 'r') as f:
        lines = f.readlines()
        miou = float(lines[-1].split(':')[-1].strip())
    return miou

def add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou):
    metrics_dict['dataset'].append(dataset)
    metrics_dict['trajectory'].append(trajectory)
    metrics_dict['label_set'].append(label_set)
    metrics_dict['seg_method'].append(seg_method)
    metrics_dict['lulc_type'].append(lulc_type)
    metrics_dict['d3_type'].append(d3_type)
    metrics_dict['res'].append(res)
    metrics_dict['refine_type'].append(refine_type)
    metrics_dict['cm'].append(cm)
    # metrics_dict['iou'].append(iou)
    metrics_dict['miou'].append(miou)

In [3]:
common_idx2names = {
    0 : 'water',
    1 : 'trees',
    2 : 'low_vegetation',
    3 : 'built',
    4 : 'ground',
    5 : 'sky',
}

more_common_idx2names = {
    0 : 'water',
    1 : 'vegetation',
    2 : 'built',
    3 : 'ground',
    4 : 'sky',
}

most_common_idx2names = {
    0 : 'water',
    1 : 'ground',
    2 : 'sky',
}

In [4]:
cm_paths = glob.glob('./outputs/*/*/*/*/*/*/*/*/confusion_matrix.npy')

In [5]:
metrics_dict = {
    'dataset' : [],
    'trajectory' : [],
    'label_set' : [],
    'seg_method' : [],
    'lulc_type' : [],
    'd3_type' : [],
    'res' : [],
    'refine_type' : [],
    'cm' : [],
    # 'iou' : [],
    'miou' : [],
}

for cm_filepath in tqdm.tqdm(cm_paths):
    iou_path = cm_filepath.replace('confusion_matrix.npy', 'metrics.txt')
    miou = read_metrics(iou_path)

    label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory = cm_filepath.split(os.path.sep)[-9:-1]
    cm = np.load(cm_filepath)
    iou = cm_to_iou(cm)
    
    add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou)
    
    # Colorado has no DSM. Use dem 1m instead.
    # Lets us compare dem vs dsm
    if dataset == '2022-05-15_ColoradoRiver' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dsm', res, refine_type, dataset, trajectory, cm, miou)

    # Lets us compare dem vs dem_1m
    if dataset == '2023-03-XX_Duck' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dem_1m', res, refine_type, dataset, trajectory, cm, miou)

    


 30%|██▉       | 3304/11046 [00:00<00:01, 6673.15it/s]/tmp/ipykernel_3682247/3340967550.py:5: RuntimeWarning: invalid value encountered in divide
  iou = tps / (tps + fps + fns)
100%|██████████| 11046/11046 [00:01<00:00, 6656.05it/s]


In [6]:
df = pd.DataFrame.from_dict(metrics_dict)

In [7]:
unique_ids = ['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']
grouped_df = df.groupby(unique_ids)

In [8]:
counts = grouped_df['miou'].transform('count')
cm_sum = grouped_df['cm'].transform('sum')
trajectory_avg_miou = grouped_df['miou'].transform('mean')

df['trajectory_count'] = counts
df['cm_sum'] = cm_sum
df['trajectory_avg_miou'] = trajectory_avg_miou
df['miou (total)'] = df.apply(lambda x: cm_to_miou(x['cm_sum']), axis=1)


In [9]:
# df_agg = df.groupby(['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']).reset_index()
df_agg_ds_stats = df.drop(columns=['dataset', 'trajectory', 'cm', 'cm_sum', 'miou'])
df_agg_ds_stats

,label_set,seg_method,lulc_type,d3_type,res,refine_type,trajectory_count,trajectory_avg_miou,miou (total)
0,most_common,open_sam_boxnms_0p35,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.791450,0.852389
1,most_common,open_sam_boxnms_0p35,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.791450,0.852389
2,most_common,open_sam_boxnms_0p35,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.791450,0.852389
3,most_common,open_sam_boxnms_0p35,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.791450,0.852389
4,most_common,open_sam_boxnms_0p35,open_earth_map_unet_lc_naip_corrected,dsm,1.0,none,11,0.791450,0.852389
...,...,...,...,...,...,...,...,...,...
13828,common,open_sam_default,dynamicworld,dem_1m,5.0,crf_naip_naip-nir,11,0.327988,0.474694
13829,common,open_sam_default,dynamicworld,dem_1m,5.0,crf_naip_naip-nir,11,0.327988,0.474694
13830,common,open_sam_default,dynamicworld,dem_1m,5.0,crf_naip_naip-nir,11,0.327988,0.474694
13831,common,open_sam_default,dynamicworld,dem_1m,5.0,crf_naip_naip-nir,11,0.327988,0.474694


In [10]:
df_agg_ds_stats.drop_duplicates(inplace=True)

In [20]:
filter_cond = (
    (df_agg_ds_stats['seg_method'] == 'open_sam_boxnms_0p50') & 
    (df_agg_ds_stats['d3_type'] == 'dsm') &
    (df_agg_ds_stats['res'] == '1.0') &
    (df_agg_ds_stats['lulc_type'] == 'dynamicworld') & 
    (df_agg_ds_stats['refine_type'] == 'crf_planet')
)

print(df_agg_ds_stats[filter_cond][['trajectory_avg_miou', 'miou (total)']].mean())
df_agg_ds_stats[filter_cond]


trajectory_avg_miou    0.550943
miou (total)           0.663254
dtype: float64


,label_set,seg_method,lulc_type,d3_type,res,refine_type,trajectory_count,trajectory_avg_miou,miou (total)
2743,most_common,open_sam_boxnms_0p50,dynamicworld,dsm,1.0,crf_planet,11,0.795315,0.875532
7354,more_common,open_sam_boxnms_0p50,dynamicworld,dsm,1.0,crf_planet,11,0.468061,0.548404
11965,common,open_sam_boxnms_0p50,dynamicworld,dsm,1.0,crf_planet,11,0.389452,0.565826


In [21]:
print(df_agg_ds_stats[filter_cond][['label_set', 'trajectory_avg_miou', 'miou (total)']].set_index('label_set').T[['common', 'more_common', 'most_common']].to_latex(float_format='{:.3f}'.format))

\begin{tabular}{lrrr}
\toprule
label_set & common & more_common & most_common \\
\midrule
trajectory_avg_miou & 0.389 & 0.468 & 0.795 \\
miou (total) & 0.566 & 0.548 & 0.876 \\
\bottomrule
\end{tabular}



In [17]:
df_agg_ds_stats['lulc_type'].unique()

array(['open_earth_map_unet_lc_naip_corrected',
       'open_earth_map_unet_lc_planet',
       'chesapeake_bay_swin_crossentropy_lc_naip_corrected',
       'chesapeake_bay_swin_crossentropy_lc_planet', 'dynamicworld'],
      dtype=object)